In [125]:
#%pip install scikit-learn
#%pip install nltk
#%pip install gensim




In [117]:
import pandas as pd
import re
import os
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
sys.path.append('../')
from utils import remove_similar_rows, find_lines_with_player, remove_similar_rows_per_player, map_emoji_to_description, del_patterns
import nltk
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import remove_stopwords, strip_numeric, strip_punctuation, strip_multiple_whitespaces




# Load Data

In [102]:
url = 'https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/data_files/all_data_v3.csv'
df = pd.read_csv(url)

In [103]:
# Filter out the Spanish data and reindex
df_es = df[df["language"] == "es"]

#Reset index
df_es = df_es.reset_index(drop=True)
df_es

,data,player,language,publishedAt
0,"{'content': ""DIRECTO\nMercado de fichajes de f...",Exequiel Palacios,es,2023-01-29T18:25:03Z
1,{'content': 'Con el primer mes del 2023 a poco...,Exequiel Palacios,es,2023-01-30T16:52:46Z
2,{'content': 'Deportes\nGustavo Puerta ya no ju...,Exequiel Palacios,es,2023-01-31T20:41:38Z
3,{'content': 'Dólar blue\nAlberto Fernández\nMa...,Exequiel Palacios,es,2023-02-09T18:32:38Z
4,{'content': 'Dólar blue\nAlberto Fernández\nMa...,Exequiel Palacios,es,2023-02-12T21:13:55Z
...,...,...,...,...
226,{'content': 'Piero Hincapié (der.) durante el ...,Piero Hincapié,es,2023-02-16T22:09:36Z
227,{'content': 'Piero Hincapié (centro) y Félix T...,Piero Hincapié,es,2023-02-14T23:40:33Z
228,"{'content': 'El DT de Ecuador Gustavo Alfaro, ...",Piero Hincapié,es,2023-02-14T15:58:38Z
229,{'content': 'Eche un vistazo a la versión beta...,Piero Hincapié,es,2023-02-03T10:58:00Z


In [37]:
print(df_es.loc[5, 'data'])

{'content': 'Una tarde particular. Una mixtura de sensaciones envolvió la antesala del juego entre River y Argentinos. La vuelta a un renovado y moderno Monumental, una arista de lo que vivenció el hincha. El homenaje a los campeones del mundo, riverplatenses que fueron artífices de las tres estrellas que suma la Argentina en la historia de la Copa del Mundo, la restante faceta de una jornada que ofreció un eje singular con la presencia de Daniel Passarella en la cancha en la que fue futbolista, director técnico y presidente del momento más oscuro de los 122 años del club. La presentación del estadio con mayor capacidad de Sudamérica, una obra que generó en el inicio protestas y hasta polémicas, tuvo nota de aprobación, mientras que la celebración y el agasajo quedó deformado entre la ovación, los aplausos, el desorden y la indiferencia.\nLa invitación a Passarella, una figura que cuando se consumó el descenso a la primera Nacional era el presidente de River, generó revuelo en el ambie

# Emojis analysis

In [7]:
# Unicode ranges for emojis
emoji_ranges = [
    (0x1F600, 0x1F64F),  # Emoticons
    (0x1F300, 0x1F5FF),  # Miscellaneous symbols and pictographs
    (0x1F680, 0x1F6FF),  # Transport and map symbols
    (0x2600, 0x26FF),    # Miscellaneous symbols
    (0x2700, 0x27BF),    # Dingbats
    (0xFE00, 0xFE0F),    # Variation Selectors
    (0x1F900, 0x1F9FF),  # Supplemental Symbols and Pictographs
    (0x1F1E6, 0x1F1FF)   # Flags
]

# Function to check if a character is an emoji
def is_emoji(character):
    for start, end in emoji_ranges:
        if ord(character) in range(start, end + 1):
            return True
    return False

def is_emoji_remove(character):
    emoji_ord = ord(character)
    for start, end in emoji_ranges:
        if emoji_ord >= start and emoji_ord <= end:
            return True
    return False

# Assuming your dataframe is named 'df'
articles_with_emoji = 0

# Iterate over the rows of the dataframe
for index, row in df_es.iterrows():
    # Counter for emojis in the current article
    emoji_count = 0

    # Iterate over the columns of the current row
    for column in row.index:
        # Check if the cell contains an emoji
        for char in str(row[column]):
            if is_emoji(char):
                emoji_count += 1

    # If there is at least one emoji in the current article, increment the count of articles with emojis
    if emoji_count > 0:
        articles_with_emoji += 1
        print("Emojis found in article", index + 1, ":", emoji_count)

print("Total number of articles with emojis:", articles_with_emoji)





Emojis found in article 6 : 6
Emojis found in article 12 : 2
Emojis found in article 14 : 4
Emojis found in article 30 : 1
Emojis found in article 32 : 1
Emojis found in article 65 : 4
Emojis found in article 78 : 2
Emojis found in article 81 : 11
Emojis found in article 15 : 11
Emojis found in article 7 : 11
Emojis found in article 11 : 12
Emojis found in article 19 : 11
Emojis found in article 21 : 1
Emojis found in article 3 : 6
Emojis found in article 4 : 3
Emojis found in article 5 : 3
Emojis found in article 9 : 1
Emojis found in article 19 : 15
Emojis found in article 1 : 5
Emojis found in article 21 : 4
Emojis found in article 26 : 20
Emojis found in article 29 : 5
Emojis found in article 30 : 3
Emojis found in article 31 : 2
Total number of articles with emojis: 24


In [104]:
df_es['data'] = df_es['data'].apply(lambda x: re.sub(r'[^\w\s]', lambda match: map_emoji_to_description(match.group(), language = 'es',), str(x)))
print(df_es.loc[9, 'data'])

{'content': 'Piero Hincapié (der.) durante el partido entre Bayer Leverkusen y Mónaco en la Europa League. Foto: @bayer04_es\nEl ecuatoriano Piero Hincapié fue titular en la visita del Bayer Leverkusen al Mónaco, en la Europa League, este 23 de febrero del 2023. A la larga, el equipo alemán se clasificó en los penales a los octavos de final del torneo.\nEl cotejo, sumado el tiempo extra, finalizó 3-2 a favor del Bayer que fue visitante ante el Mónaco en el Estadio Luis II\u200b del distrito Fontvieille, en el Principado de Mónaco.\nLa semana pasada, el Mónaco se impuso de visitante con el mismo marcador.\nFue así que el encuentro de este 23 de febrero del 2023 se fue a los alargues y luego se definió en penales. Esa tanda favoreció al elenco del ecuatoriano.\n"Avanzamos en la Europa League, bien hecho leones" fue el mensaje que publicó el cuadro alemán en las redes sociales.\nLos goles del Leverkusen los anotaron  Florian Wirtz, Exequiel Palacios y Amine Adli.\nPor el elenco local anot

# Remove Patterns

In [105]:
patternlist = [
    'content',
    'DIRECTO',
    'Espacio Publicitario',
    'Copyright',
    'copyright',
    'Foto:',
    'Todos los derechos reservados',
    'Derechos reservados',
    'Suscribete',
    'Queda prohibida la reproducción',
    'parcial, por',
    'cualquier medio, de todos los contenidos sin autorización expresa de Grupo EL COMERCIO',
    'pic.twitter.com'
   
]

In [106]:
# Create a copy
data_wo_pattern = df_es.copy()

data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: del_patterns(str(x), patternlist))

In [107]:

#link
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r'http\S+', '', str(x)))

# strip multiple whitespaces also \n -> (https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.strip_multiple_whitespaces)
data_wo_pattern['data'] = data_wo_pattern['data'].apply(strip_multiple_whitespaces)

# Remove the similiar rows (The Function is imported from utils on top)
data_wo_pattern = remove_similar_rows_per_player(data_wo_pattern, data_wo_pattern['player'].unique())

# Reset index
data_wo_pattern.reset_index(drop=True, inplace=True)




# Find line with player

In [108]:
data_with_playernames = data_wo_pattern.copy()

data_with_playernames = find_lines_with_player(data_with_playernames, data_with_playernames['player'].unique())

# Reset index
data_with_playernames.reset_index(drop=True, inplace=True)

In [109]:
print(data_with_playernames.loc[2, 'data'])

Gustavo Puerta ya no jugará en el Bayer LeverkusenEntérate en qué equipo de la segunda división jugará Gustavo Puerta, capitán de la Selección Colombia Sub-20, que participa en el Sudamericano.Por: John Brandon García TeheránEspecial para El UniversalA pocas horas del hexagonal final del Sudamericano Sub-20, que se realiza en Colombia, la Selección Nacional disputará un partido trascendental para su continuidad en el campeonato frente a Uruguay.Bayer Leverkusen pescó a una de las figuras mas importantes del Sudamericano Sub-20, Gustavo Puerta. El club alemán, que siempre se ha caracterizado por confiar en gente joven con sangre nueva y, por supuesto, en mediocampistas sudamericanos con sacrificio, físico y bien dotados técnicamente, como lo son el chileno Charles Aranguiz que actualmente se encuentra lesionado y su regreso se espera para mitad de febrero, y el Argentino de 24 años Exequiel Palacios. Le puede interesar: Video: Se quitó el yeso, entró y le dio el gol del triunfo a Bolíva

In [119]:
# Create the folder if it doesn't exist
folder_name = 'data_clean'

# Define the file path for saving the CSV
file_name = 'es_clean_1.csv'
file_path = os.path.join(folder_name, file_name)

# Convert the dataframe to CSV and save it
data_with_playernames.to_csv(file_path, index=False)

# Stop words

In [113]:
#Create copy to new df data_rm_sw as in data removed stopwords
data_es_rm = data_with_playernames.copy()

#Load stopwords
#nltk.download('stopwords')
spanish_stopwords = stopwords.words('spanish')


In [114]:
# Create a list of stop words to remove 
stop_words_to_remove = ['ni', 'no']

# Remove the stop words to remove from the stop words list
for word in stop_words_to_remove:
  spanish_stopwords.remove(word)

# Define a function to apply remove_stopwords on a column
def remove_stopwords_from_text(text):
    return remove_stopwords(text, stopwords=spanish_stopwords)

# Apply the remove_stopwords function to the 'text' column using the apply method
data_es_rm['data'] = data_es_rm['data'].apply(remove_stopwords_from_text)



# More removals

In [115]:
# strip_numeric -> removing digits (https://gensimr.news-r.org/reference/strip_numeric.html)
data_es_rm['data'] = data_es_rm['data'].apply(strip_numeric)

# strip_punctutation -> removing punctations (https://gensimr.news-r.org/reference/strip_punctuation.html)
data_es_rm['data'] = data_es_rm['data'].apply(strip_punctuation)

# strip multiple whitespaces also \n -> (https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.strip_multiple_whitespaces)
data_es_rm['data'] = data_es_rm['data'].apply(strip_multiple_whitespaces)

# Convert text to lowercase
data_es_rm['data'] = data_es_rm['data'].str.lower()


# Reset index
data_es_rm.reset_index(drop=True, inplace=True)



In [118]:
# Create the folder if it doesn't exist
folder_name = 'data_clean'

# Define the file path for saving the CSV
file_name = 'es_clean_2.csv'
file_path = os.path.join(folder_name, file_name)

# Convert the dataframe to CSV and save it
data_es_rm.to_csv(file_path, index=False)


